Тестовый запуск модели

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

In [ ]:
!git clone https://github.com/xinntao/ESRGAN
!cd ESRGAN

Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 225 (delta 15), reused 14 (delta 14), pack-reused 205 (from 1)
Receiving objects: 100% (225/225), 24.86 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
import os

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import ESRGAN.RRDBNet_arch as arch

In [ ]:


model_path = 'ESRGAN/models/RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
#device = torch.device('cpu')

test_img_folder = 'ESRGAN/LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('ESRGAN/results/{:s}_rlt.png'.format(base), output)

Model path ESRGAN/models/RRDB_ESRGAN_x4.pth. 
Testing...
1 baboon
2 comic


Запускаем на датасете

In [ ]:
import os
import zipfile
import random
import requests

# --- Настройки ---
output_dir = "output"
hr_folder = os.path.join(output_dir, "HR")
lr_folder = os.path.join(output_dir, "LR")

hr_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip"
lr_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip"

hr_zip_path = "DIV2K_train_HR.zip"
lr_zip_path = "DIV2K_train_LR_bicubic_X2.zip"

# --- Создание папок ---
os.makedirs(hr_folder, exist_ok=True)
os.makedirs(lr_folder, exist_ok=True)

# --- Скачивание архива (если не скачан) ---
def download_file(url, path):
    if not os.path.exists(path):
        print(f"Скачиваю {url} ...")
        r = requests.get(url, stream=True)
        with open(path, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
        print(f"Скачано: {path}")
    else:
        print(f"Уже скачан: {path}")

# --- Скачиваем архивы ---
download_file(hr_url, hr_zip_path)
download_file(lr_url, lr_zip_path)

# --- Случайные 5 индексов (800-900) ---
selected_indices = random.sample(range(800, 901), 5)
print("Выбранные изображения:", selected_indices)

# --- Распаковка выборочно ---
def extract_selected(zip_path, file_template, indices, output_folder):
    with zipfile.ZipFile(zip_path, 'r') as z:
        for idx in indices:
            filename = file_template.format(idx)
            if filename in z.namelist():
                z.extract(filename, output_folder)
                print(f"Извлечено: {filename}")
            else:
                print(f"Файл не найден в архиве: {filename}")

# --- Распаковываем HR ---
extract_selected(
    zip_path=hr_zip_path,
    file_template="DIV2K_train_HR/{:04d}.png",
    indices=selected_indices,
    output_folder=hr_folder
)

# --- Распаковываем LR x2 ---
extract_selected(
    zip_path=lr_zip_path,
    file_template="DIV2K_train_LR_bicubic/X2/{:04d}x2.png",
    indices=selected_indices,
    output_folder=lr_folder
)

Скачиваю http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip ...
Скачано: DIV2K_train_HR.zip
Скачиваю http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip ...
Скачано: DIV2K_train_LR_bicubic_X2.zip
Выбранные изображения: [838, 848, 899, 843, 830]
Файл не найден в архиве: DIV2K_train_HR/0838.png
Файл не найден в архиве: DIV2K_train_HR/0848.png
Файл не найден в архиве: DIV2K_train_HR/0899.png
Файл не найден в архиве: DIV2K_train_HR/0843.png
Файл не найден в архиве: DIV2K_train_HR/0830.png
Файл не найден в архиве: DIV2K_train_LR_bicubic/X2/0838x2.png
Файл не найден в архиве: DIV2K_train_LR_bicubic/X2/0848x2.png
Файл не найден в архиве: DIV2K_train_LR_bicubic/X2/0899x2.png
Файл не найден в архиве: DIV2K_train_LR_bicubic/X2/0843x2.png
Файл не найден в архиве: DIV2K_train_LR_bicubic/X2/0830x2.png


Делаем для видео

In [ ]:
!pip install opencv-python numpy

In [ ]:
import os
import cv2
import glob
import numpy as np
import torch
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
import ESRGAN.archs.rrdbnet_arch as arch  # Путь к архитектуре RRDBNet

In [ ]:
model_path = 'ESRGAN/models/RRDB_ESRGAN_x4.pth'
device = torch.device('cpu')  # или 'cuda', если доступна
input_video_path = 'cat_murr_and_meow.mp4'
output_video_path = 'cat_murr_and_meow_upscaled.mp4'
temp_frames_dir = 'ESRGAN/temp_frames/'
results_dir = 'ESRGAN/results_video/'
os.makedirs(temp_frames_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [ ]:

# === ЗАГРУЗКА МОДЕЛИ ===
model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path, map_location=device), strict=True)
model.eval()
model = model.to(device)

In [ ]:
# === ИЗВЛЕЧЕНИЕ КАДРОВ ИЗ ВИДЕО ===
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_paths = []

frame_idx = 0
print("Извлечение кадров...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(temp_frames_dir, f"frame_{frame_idx:04d}.png")
    cv2.imwrite(frame_path, frame)
    frame_paths.append(frame_path)
    frame_idx += 1
cap.release()

Извлечение кадров...


In [ ]:
# === АПСКЕЙЛ КАДРОВ С ПОМОЩЬЮ ESRGAN ===
print("Апскейл кадров...")
for path in tqdm(frame_paths):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)
    cv2.imwrite(os.path.join(results_dir, f"{base}_upscaled.png"), output)

# === СОБИРАНИЕ ВИДЕО ОБРАТНО ===
upscaled_frames = sorted(glob.glob(os.path.join(results_dir, '*_upscaled.png')))
sample_frame = cv2.imread(upscaled_frames[0])
height, width, _ = sample_frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

print("Сборка видео...")
for frame_path in upscaled_frames:
    frame = cv2.imread(frame_path)
    out.write(frame)
out.release()

print(f"✅ Готово! Апскейленное видео сохранено как: {output_video_path}")

Апскейл кадров...


100%|██████████| 111/111 [3:19:36<00:00, 107.90s/it]


Сборка видео...
✅ Готово! Апскейленное видео сохранено как: cat_murr_and_meow_upscaled.mp4
